In [1]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
from scipy.sparse import csr_matrix
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time
import argparse
import random
import csv

In [10]:
def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=1000, type=int)
    parser.add_argument("--number-of-clauses", default=100, type=int)
    parser.add_argument("--T", default=np.power((1000/0.8377),1/2.2099), type=int)
    parser.add_argument("--s", default=2.534*np.log((1000/3.7579)), type=float)
    parser.add_argument("--depth", default=2, type=int)
    parser.add_argument("--hypervector-size", default=32, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=256, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument('--double-hashing', dest='double_hashing', default=False, action='store_true')
    parser.add_argument("--noise", default=0.01, type=float)
    parser.add_argument("--number-of-examples", default=10000, type=int)
    parser.add_argument("--max-included-literals", default=4, type=int)

    args, unknown = parser.parse_known_args()
    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    return args

args = default_args()

In [3]:
print("Loading data")
input_file = 'hex_games_1_000_000_size_7.csv'

# Read the entire CSV file into a list of rows
with open(input_file, mode='r', newline='') as infile:
    reader = csv.reader(infile)
    data = list(reader)


delete_index = int(0.995 * len(data))
data = data[delete_index:]
# Split the data into training (80%) and testing (20%) sets
split_index = int(0.8 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

# Separate X_data and Y_data for training set
X_train = np.array([row[:-1] for row in train_data], dtype=int)
Y_train = np.array([row[-1:] for row in train_data], dtype=int)

# Separate X_data and Y_data for testing set
X_test = np.array([row[:-1] for row in test_data], dtype=int)
Y_test = np.array([row[-1:] for row in test_data], dtype=int)

X_train = X_train.reshape(-1, 7, 7)
X_test = X_test.reshape(-1, 7, 7)

Y_train = Y_train.reshape(-1)
Y_test = Y_test.reshape(-1)

# Print the shapes of the arrays
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


Loading data
X_train shape: (4000, 7, 7)
Y_train shape: (4000,)
X_test shape: (1000, 7, 7)
Y_test shape: (1000,)


In [5]:
print("Creating training data")

# Create train data

graphs_train = Graphs(
    X_train.shape[0],
    symbols=['A', 'B', 'O'],
    hypervector_size=args.hypervector_size,
    hypervector_bits=args.hypervector_bits,
)

patch_size = 1
dim = 7 - patch_size + 1

number_of_nodes = dim * dim

print(patch_size, dim, number_of_nodes)

for graph_id in range(X_train.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, number_of_nodes)
    
graphs_train.prepare_node_configuration()
print("Finished preparing node configuration")

map_size = 7 # 7 x 7
data_size = 10000
directions = [
    (-1, 0), (1, 0),  # Left and Right
    (0, -1), (0, 1),  # Up and Down
    (-1, 1), (1, -1)  # Upper-left and Lower-right for staggered rows
]

for graph_id in range(X_train.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            number_of_outgoing_edges = 0
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy
                
                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                    number_of_outgoing_edges += 1
            # Use the node_counter to create a unique label for each node
            graphs_train.add_graph_node(graph_id, node_id, number_of_outgoing_edges)
            
    
print("All nodes are acounted for")
graphs_train.prepare_edge_configuration()
edge_type = "Plain"

for graph_id in range(X_train.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            number_of_outgoing_edges = 0
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy

                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                    neighbor_id = neighbor_y * map_size + neighbor_x
                    graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id, edge_type)
print("Added neighbor edges")


Creating training data
1 7 49
Finished preparing node configuration
All nodes are acounted for
Added neighbor edges


In [6]:
print("Preparing to add briks to the board")
for graph_id in range(X_train.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            
            if X_train[graph_id][y][x] == 1:
                brick = 'A'
            elif X_train[graph_id][y][x] == -1:
                brick = 'B'
            else:
                brick = 'O'
            graphs_train.add_graph_node_property(graph_id, node_id, brick)

graphs_train.encode()
print("All cells has now a brick")


Preparing to add briks to the board
All cells has now a brick


In [7]:
print("Creating training data")

# Create train data

graphs_test = Graphs(
    X_test.shape[0],
    symbols=['A', 'B', 'O'],
    hypervector_size=args.hypervector_size,
    hypervector_bits=args.hypervector_bits,
)

patch_size = 1
dim = 7 - patch_size + 1

number_of_nodes = dim * dim

print(patch_size, dim, number_of_nodes)

for graph_id in range(X_test.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, number_of_nodes)
    
graphs_test.prepare_node_configuration()
print("Finished preparing node configuration")

map_size = 7 # 7 x 7
data_size = 10000
directions = [
    (-1, 0), (1, 0),  # Left and Right
    (0, -1), (0, 1),  # Up and Down
    (-1, 1), (1, -1)  # Upper-left and Lower-right for staggered rows
]

for graph_id in range(X_test.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            number_of_outgoing_edges = 0
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy
                
                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                    number_of_outgoing_edges += 1

            # Use the node_counter to create a unique label for each node
            graphs_test.add_graph_node(graph_id, node_id, number_of_outgoing_edges)
            


print("All nodes are acounted for")
graphs_test.prepare_edge_configuration()
edge_type = "Plain"

for graph_id in range(X_test.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            number_of_outgoing_edges = 0
    
            
            for dx, dy in directions:
                neighbor_x = x + dx
                neighbor_y = y + dy

                # Check if neighbor exists within bounds
                if 0 <= neighbor_x < map_size and 0 <= neighbor_y < map_size:
                    neighbor_id = neighbor_y * map_size + neighbor_x
                    graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id, edge_type)
print("Added neighbor edges")



Creating training data
1 7 49
Finished preparing node configuration
All nodes are acounted for
Added neighbor edges


In [8]:
print("Preparing to add briks to the board")
for graph_id in range(X_test.shape[0]):
    for x in range(map_size):
        for y in range(map_size):
            node_id = y * map_size + x
            
            if X_test[graph_id][y][x] == 1:
                brick = 'A'
            elif X_test[graph_id][y][x] == -1:
                brick = 'B'
            else:
                brick = 'O'
            graphs_test.add_graph_node_property(graph_id, node_id, brick)

graphs_test.encode()
print("All cells has now a brick")


Preparing to add briks to the board
All cells has now a brick


In [ ]:
tm = MultiClassGraphTsetlinMachine(
    args.number_of_clauses,
    args.T,
    args.s,
    depth = args.depth,
    message_size = args.message_size,
    message_bits = args.message_bits,
    max_included_literals = args.max_included_literals
)

for i in range(args.epochs):
    start_training = time()
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time()

    start_testing = time()
    result_test = 100*(tm.predict(graphs_test) == Y_test).mean()
    stop_testing = time()

    result_train = 100*(tm.predict(graphs_train) == Y_train).mean()

    print("%d %.2f %.2f %.2f %.2f" % (i, result_train, result_test, stop_training-start_training, stop_testing-start_testing))

weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(args.hypervector_size * 2):
            if tm.ta_action(0, i, k):
                if k < args.hypervector_size:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - args.hypervector_size))

        # for k in range(args.message_size * 2):
        #     if tm.ta_action(1, i, k):
        #         if k < args.message_size:
        #             l.append("c%d" % (k))
        #         else:
        #             l.append("NOT c%d" % (k - args.message_size))

        print(" AND ".join(l))

print(graphs_test.hypervectors)
print(tm.hypervectors)
print(graphs_test.edge_type_id)

Initialization of sparse structure.
0 0.00 51.90 3.91 0.15
1 0.00 42.40 1.19 0.16
2 0.00 0.00 1.21 0.15
3 0.00 0.00 1.24 0.15
4 0.00 0.00 1.15 0.13
5 0.00 0.00 1.14 0.13
6 0.00 0.00 1.16 0.14
7 0.00 0.00 1.16 0.13
8 0.00 0.00 1.20 0.14
9 0.00 0.00 1.16 0.16
10 0.00 0.00 1.19 0.14
11 0.00 0.00 1.17 0.13
12 0.00 0.00 1.15 0.13
13 0.00 0.00 1.16 0.13
14 0.00 0.00 1.15 0.13
15 0.00 0.00 1.14 0.13
16 0.00 0.00 1.15 0.13
17 0.00 0.00 1.15 0.13
18 0.00 0.00 1.18 0.13
19 0.00 0.00 1.18 0.13
20 0.00 0.00 1.15 0.14
21 0.00 0.00 1.17 0.13
22 0.00 0.00 1.17 0.13
23 0.00 0.00 1.16 0.13
24 0.00 0.00 1.21 0.13
25 0.00 0.00 1.14 0.13
26 0.00 0.00 1.14 0.13
27 0.00 0.00 1.16 0.13
28 0.00 0.00 1.15 0.13
29 0.00 0.00 1.14 0.14
30 0.00 0.00 1.14 0.13
31 0.00 0.00 1.14 0.13
32 0.00 0.00 1.16 0.15
